In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import folium
from shapely import wkt

# 횡단보도 위치 데이터 읽기 및 전처리
crosswalk_df = pd.read_csv('/content/drive/MyDrive/수삼/서울시 대로변 횡단보도 위치정보 (1).csv', encoding='cp949')

# 파리바게트 위치 데이터 읽기
df = pd.read_csv('/content/drive/MyDrive/수삼/소상공인시장진흥공단_상가(상권)정보_20240331/소상공인시장진흥공단_상가(상권)정보_서울_202403.csv')
df_seoul = df[df['시도명'] == '서울특별시'].copy()
df_seoul["상호명_소문자"] = df_seoul["상호명"].str.lower()
df_parisbaguette = df_seoul[df_seoul["상호명_소문자"].str.contains("(파|빠)리바게트|파리 바게트|parisbaguette")].copy()

# 횡단보도 데이터 샘플링
sample_size = min(100000, len(crosswalk_df))  # 횡단보도 데이터 전체를 사용할 수 있도록 수정
df_seoul_crosswalk_sample = crosswalk_df.sample(n=sample_size, random_state=1)

# Folium 지도 생성
seoul_map = folium.Map(location=[37.5665, 126.978], zoom_start=12)

# 파리바게트 위치 마커 추가
for idx, row in df_parisbaguette.iterrows():
    folium.Marker([row['위도'], row['경도']], popup='파리바게트', icon=folium.Icon(color='blue')).add_to(seoul_map)

# 횡단보도 위치 마커 추가 (NaN 값 처리)
df_seoul_crosswalk_sample = df_seoul_crosswalk_sample.dropna(subset=['노드 WKT'])  # NaN 값이 있는 행 제거

for idx, row in df_seoul_crosswalk_sample.iterrows():
    node = wkt.loads(row['노드 WKT'])
    latitude = node.y
    longitude = node.x
    folium.Marker([latitude, longitude], popup='횡단보도', icon=folium.Icon(color='orange')).add_to(seoul_map)

# 지도 출력 및 저장
seoul_map.save('/content/drive/MyDrive/수삼/seoul_map.html')
seoul_map

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
import pandas as pd
import folium
from shapely import wkt
from geopy.distance import geodesic

# 횡단보도 위치 데이터 읽기 및 전처리
crosswalk_df = pd.read_csv('/content/drive/MyDrive/수삼/서울시 대로변 횡단보도 위치정보 (1).csv', encoding='cp949')

# 파리바게트 위치 데이터 읽기
df = pd.read_csv('/content/drive/MyDrive/수삼/소상공인시장진흥공단_상가(상권)정보_20240331/소상공인시장진흥공단_상가(상권)정보_서울_202403.csv')
df_seoul = df[df['시도명'] == '서울특별시'].copy()
df_seoul["상호명_소문자"] = df_seoul["상호명"].str.lower()
df_parisbaguette = df_seoul[df_seoul["상호명_소문자"].str.contains("(파|빠)리바게트|파리 바게트|parisbaguette")].copy()

# 횡단보도 데이터 샘플링
sample_size = min(100000, len(crosswalk_df))  # 횡단보도 데이터 전체를 사용할 수 있도록 수정
df_seoul_crosswalk_sample = crosswalk_df.sample(n=sample_size, random_state=1)

# Folium 지도 생성
seoul_map = folium.Map(location=[37.5665, 126.978], zoom_start=12)

# 파리바게트 위치 마커 추가
for idx, row in df_parisbaguette.iterrows():
    folium.Marker([row['위도'], row['경도']], popup='파리바게트', icon=folium.Icon(color='blue')).add_to(seoul_map)

# 횡단보도 위치 마커 추가 (NaN 값 처리)
df_seoul_crosswalk_sample = df_seoul_crosswalk_sample.dropna(subset=['노드 WKT'])  # NaN 값이 있는 행 제거

# 파리바게트 근처 100m 이내의 횡단보도 표시
for idx, paris_row in df_parisbaguette.iterrows():
    paris_location = (paris_row['위도'], paris_row['경도'])

    for idx, crosswalk_row in df_seoul_crosswalk_sample.iterrows():
        try:
            node = wkt.loads(crosswalk_row['노드 WKT'])
            crosswalk_location = (node.y, node.x)

            # 두 지점 간의 거리 계산 (단위: 미터)
            dist = geodesic(paris_location, crosswalk_location).meters

            if dist <= 100:
                folium.Marker([crosswalk_location[0], crosswalk_location[1]], popup='횡단보도', icon=folium.Icon(color='orange')).add_to(seoul_map)
        except Exception as e:
            print(f"Error processing row: {crosswalk_row}, Error: {e}")

# 지도 출력 및 저장
seoul_map.save('/content/drive/MyDrive/수삼/seoul_map_with_crosswalks_near_parisbaguette.html')
seoul_map

<ipython-input-16-27b8decab9cb>:10: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/drive/MyDrive/수삼/소상공인시장진흥공단_상가(상권)정보_20240331/소상공인시장진흥공단_상가(상권)정보_서울_202403.csv')
<ipython-input-16-27b8decab9cb>:13: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_parisbaguette = df_seoul[df_seoul["상호명_소문자"].str.contains("(파|빠)리바게트|파리 바게트|parisbaguette")].copy()


In [ ]:
import pandas as pd
import folium
from shapely import wkt
from geopy.distance import geodesic

# 횡단보도 위치 데이터 읽기 및 전처리
crosswalk_df = pd.read_csv('/content/drive/MyDrive/수삼/서울시 대로변 횡단보도 위치정보 (1).csv', encoding='cp949')

# 파리바게트 위치 데이터 읽기
df = pd.read_csv('/content/drive/MyDrive/수삼/소상공인시장진흥공단_상가(상권)정보_20240331/소상공인시장진흥공단_상가(상권)정보_서울_202403.csv')
df_seoul = df[df['시도명'] == '서울특별시'].copy()
df_seoul["상호명_소문자"] = df_seoul["상호명"].str.lower()
df_parisbaguette = df_seoul[df_seoul["상호명_소문자"].str.contains("(파|빠)리바게트|파리 바게트|parisbaguette")].copy()

# 횡단보도 데이터 샘플링
sample_size = min(100000, len(crosswalk_df))  # 횡단보도 데이터 전체를 사용할 수 있도록 수정
df_seoul_crosswalk_sample = crosswalk_df.sample(n=sample_size, random_state=1)

# Folium 지도 생성
seoul_map = folium.Map(location=[37.5665, 126.978], zoom_start=12)

# 횡단보도 위치 마커 추가 (NaN 값 처리)
df_seoul_crosswalk_sample = df_seoul_crosswalk_sample.dropna(subset=['노드 WKT'])  # NaN 값이 있는 행 제거

# 파리바게트 근처 100m 이내의 횡단보도 표시 및 100m 이내에 횡단보도가 없는 파리바게트 표시
for idx, paris_row in df_parisbaguette.iterrows():
    paris_location = (paris_row['위도'], paris_row['경도'])
    near_crosswalk = False

    for idx, crosswalk_row in df_seoul_crosswalk_sample.iterrows():
        try:
            node = wkt.loads(crosswalk_row['노드 WKT'])
            crosswalk_location = (node.y, node.x)

            # 두 지점 간의 거리 계산 (단위: 미터)
            dist = geodesic(paris_location, crosswalk_location).meters

            if dist <= 100:
                near_crosswalk = True
                folium.Marker([crosswalk_location[0], crosswalk_location[1]], popup='횡단보도', icon=folium.Icon(color='orange')).add_to(seoul_map)
        except Exception as e:
            print(f"Error processing row: {crosswalk_row}, Error: {e}")

    if not near_crosswalk:
        folium.Marker([paris_row['위도'], paris_row['경도']], popup='파리바게트', icon=folium.Icon(color='red')).add_to(seoul_map)
    else:
        folium.Marker([paris_row['위도'], paris_row['경도']], popup='파리바게트', icon=folium.Icon(color='blue')).add_to(seoul_map)

# 지도 출력 및 저장
seoul_map.save('/content/drive/MyDrive/수삼/seoul_map_with_crosswalks_near_parisbaguette.html')
seoul_map

<ipython-input-1-a5922fd03bcb>:10: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/drive/MyDrive/수삼/소상공인시장진흥공단_상가(상권)정보_20240331/소상공인시장진흥공단_상가(상권)정보_서울_202403.csv')
<ipython-input-1-a5922fd03bcb>:13: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_parisbaguette = df_seoul[df_seoul["상호명_소문자"].str.contains("(파|빠)리바게트|파리 바게트|parisbaguette")].copy()


In [ ]:
import pandas as pd
import folium
from shapely import wkt
from geopy.distance import geodesic

# 횡단보도 위치 데이터 읽기 및 전처리
crosswalk_df = pd.read_csv('/content/drive/MyDrive/수삼/서울시 대로변 횡단보도 위치정보 (1).csv', encoding='cp949')

# 파리바게트 위치 데이터 읽기
df = pd.read_csv('/content/drive/MyDrive/수삼/소상공인시장진흥공단_상가(상권)정보_20240331/소상공인시장진흥공단_상가(상권)정보_서울_202403.csv')
df_seoul = df[df['시도명'] == '서울특별시'].copy()
df_seoul["상호명_소문자"] = df_seoul["상호명"].str.lower()
df_parisbaguette = df_seoul[df_seoul["상호명_소문자"].str.contains("(파|빠)리바게트|파리 바게트|parisbaguette")].copy()

# 횡단보도 데이터 샘플링
sample_size = min(100000, len(crosswalk_df))  # 횡단보도 데이터 전체를 사용할 수 있도록 수정
df_seoul_crosswalk_sample = crosswalk_df.sample(n=sample_size, random_state=1)

# Folium 지도 생성
seoul_map = folium.Map(location=[37.5665, 126.978], zoom_start=12)

# 횡단보도 위치 마커 추가 (NaN 값 처리)
df_seoul_crosswalk_sample = df_seoul_crosswalk_sample.dropna(subset=['노드 WKT'])  # NaN 값이 있는 행 제거

# 파리바게트 근처 100m 이내의 횡단보도 표시 및 100m 이내에 횡단보도가 없는 파리바게트 표시
red_icon_count = 0  # 빨간색 아이콘의 파리바게트 개수

for idx, paris_row in df_parisbaguette.iterrows():
    paris_location = (paris_row['위도'], paris_row['경도'])
    near_crosswalk = False

    for idx, crosswalk_row in df_seoul_crosswalk_sample.iterrows():
        try:
            node = wkt.loads(crosswalk_row['노드 WKT'])
            crosswalk_location = (node.y, node.x)

            # 두 지점 간의 거리 계산 (단위: 미터)
            dist = geodesic(paris_location, crosswalk_location).meters

            if dist <= 100:
                near_crosswalk = True
                folium.Marker([crosswalk_location[0], crosswalk_location[1]], popup='횡단보도', icon=folium.Icon(color='orange')).add_to(seoul_map)
        except Exception as e:
            print(f"Error processing row: {crosswalk_row}, Error: {e}")

    if not near_crosswalk:
        folium.Marker([paris_row['위도'], paris_row['경도']], popup='파리바게트', icon=folium.Icon(color='red')).add_to(seoul_map)
        red_icon_count += 1  # 빨간색 아이콘 카운트 증가
    else:
        folium.Marker([paris_row['위도'], paris_row['경도']], popup='파리바게트', icon=folium.Icon(color='blue')).add_to(seoul_map)

# 빨간색 아이콘의 파리바게트 개수 텍스트로 지도에 추가
folium.map.Marker(
    [37.5665, 126.978],
    icon=folium.DivIcon(
        html=f'<div style="font-size: 20px; color: red;">빨간색 파리바게트 개수: {red_icon_count}</div>'
    )
).add_to(seoul_map)

# 지도 출력 및 저장
seoul_map.save('/content/drive/MyDrive/수삼/seoul_map_with_crosswalks_near_parisbaguette.html')
seoul_map

<ipython-input-3-eab7014cfec3>:10: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/drive/MyDrive/수삼/소상공인시장진흥공단_상가(상권)정보_20240331/소상공인시장진흥공단_상가(상권)정보_서울_202403.csv')
<ipython-input-3-eab7014cfec3>:13: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_parisbaguette = df_seoul[df_seoul["상호명_소문자"].str.contains("(파|빠)리바게트|파리 바게트|parisbaguette")].copy()
